In [1]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from IPython.display import display, HTML
from typing import List
from mprompt.modules.emb_diff_module import EmbDiffModule
import numpy as np
import matplotlib
import imodelsx.util
from copy import deepcopy
import re
import mprompt.viz
import scipy.special
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from mprompt.methods.m4_evaluate import D5_Validator
import openai
from mprompt.modules.fmri_module import fMRIModule

openai.api_key_path = os.path.expanduser('~/.OPENAI_KEY')


r = (pd.read_pickle('../results/results_fmri.pkl')
    .sort_values(by=['top_score_synthetic'], ascending=False))

### Select voxels

In [2]:
# expls = ['baseball','animals','water','movement','religion','time','technology']
# interesting_expls = ['food', 'numbers', 'physical contact', 'time', 'laughter', 'age', 'clothing']
# voxels = [('movement', 'UTS01',	7), ('numbers', 'UTS03', 55), ('time', 'UTS03', 19), ('relationships', 'UTS01', 21),
        #   ('sounds', 'UTS03', 35), ('emotion', 'UTS03', 23), ('food', 'UTS03', 46)]
# voxels = [('numbers', 'UTS03', 55), ('time', 'UTS03', 19),
        #   ('sounds', 'UTS03', 35), ('emotion', 'UTS03', 23), ('food', 'UTS03', 46)]
voxels = [('movement', 'UTS01',	7),('relationships', 'UTS01', 21)]

In [3]:
# display all rows
with pd.option_context('display.max_rows', None):
    display(r.sort_values(by=['top_score_synthetic'])[['subject', 'module_num', 'top_explanation_init_strs', 'top_ngrams_module_correct']])

,subject,module_num,top_explanation_init_strs,top_ngrams_module_correct
3,UTS01,7,movement or action,"[turned and walked, stopped midstride, slowly ..."
1,UTS03,55,numbers,"[factor of twenty, hour and twenty, hundred an..."
5,UTS03,19,passing of time,"[the days passed, weeks became months, time we..."
2,UTS01,21,relationships,"[felt like brothers, us like family, friends c..."
0,UTS03,35,sound or noise,"[hear boys laughing, i provoked gasps, she sta..."
4,UTS03,23,emotional reactions,"[runs out crying, burst into tears, she starte..."
6,UTS03,46,food items,"[cucumber and mayonnaise, some sliced cucumber..."


In [4]:
# manually check that we got the right voxels
rows = []
for vox in voxels:
    expl, subj, vox_num = vox
    rows.append(r[(r.subject == subj) & (r.module_num == vox_num)].iloc[0])
rows_check = pd.DataFrame(rows)
rows_check['expl'] = [v[0] for v in voxels]

with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(rows_check[['subject', 'module_num', 'top_explanation_init_strs', 'top_ngrams_module_correct']])

,subject,module_num,top_explanation_init_strs,top_ngrams_module_correct
3,UTS01,7,movement or action,"[turned and walked, stopped midstride, slowly stand back, i stopped midstride, turned and said, to stop halfway, turn and look, hitch a ride, called her back, and darted back, extended a hand, cut him out, moving in together, wrote them letters, turn around and, and quickly disappeared, his back turned, extended my hands, reached over and, write another letter, embraced me, to look down, darted to safety, calls me later, i moved out, sneak back, i stepped back, darted back, texted me twice, i stood back, called me up, look down and, disappeared from view, run away and, just stood back, cut her off, suddenly he stops, answered my calls, approached me, she shouted back, she turned and, sneak away, skype the unboyfriend, step back and, looks up and]"
2,UTS01,21,relationships,"[felt like brothers, us like family, friends come over, know me friends, we were engaged, friends n shit, husband and ask, my boyfriend from, with other women, girlfriend now ex, be our family, just friends, my wife from, friend from home, friends and now, friends and shit, stalemate we kissed, friends come, friends again and, like brothers like, sister and we, asked me out, brother who sometimes, be my husband, girlfriend coming, be just friends, wife random girls, wife but we, sister and now, friends from, husband now we, friends back, me my friends, brother or your, ex girlfriend from, friend through, my fiance of, boyfriend type person, love younger men, friend to me, mom and siblings, sister all, we broke up, were family we, my black friends, friends and lovers, friends seemed like, friends on, family and we, become my friend, friends and mostly, already friends and, husband who was, we became closer, married my brother, marry me until, boyfriend tonight, friend from, sister and both, pals that were, friends were were]"


# Generate story

In [ ]:
# get a list of prompts
expls = [v[0] for v in voxels]
def get_prompts_basic(expls):
    prompt_init = 'Write the beginning paragraph of a story about "{expl}". Make sure it contains several references to "{expl}".'
    prompt_continue = 'Write the next paragraph of the story, but now make it about "{expl}". Make sure it contains several references to "{expl}".'
    prompts = [prompt_init.format(expl=expls[0])] + [prompt_continue.format(expl=expl) for expl in expls[1:]]
    return prompts
prompts = get_prompts_basic(expls)
for p in prompts:
    print(p)

In [ ]:
# generate the paragraphs
paragraphs = mprompt.llm.get_paragraphs(prompts)
for para in paragraphs:
    print(para)

# Test synthetic data contains concept

In [ ]:
# mod = EmbDiffModule()
val = D5_Validator()

### Visualize single story

In [ ]:
story_running = ''
scores = {}
for i in range(len(expls)):
# for i in range(1):
    expl = expls[i].lower()
    text = paragraphs[i]
    words = text.split()
    prompt = prompts[i]

    ngrams = imodelsx.util.generate_ngrams_list(text.lower(), ngrams=3)
    ngrams = [words[0], words[0] + ' ' + words[1]] + ngrams

    # # embdiff-based viz
    # mod._init_task(expl)    
    # neg_dists = mod(ngrams)
    # assert len(ngrams) == len(words) == len(neg_dists)
    # # neg_dists = scipy.special.softmax(neg_dists)
    # # plt.plot(neg_dists)
    # # plt.plot(moving_average(neg_dists, n=5))
    # neg_dists = moving_average(neg_dists, n=3)
    # neg_dists = (neg_dists - neg_dists.min()) / (neg_dists.max() - neg_dists.min())
    # neg_dists = neg_dists / 2 + 0.5 # shift to 0.5-1 range
    # s = mprompt.viz.colorize(words, neg_dists, title=expl, subtitle=prompt)

    # validator-based viz
    probs = np.array(val.validate_w_scores(expl, ngrams))
    probs_disp = mprompt.viz.moving_average(probs, n=3)
    probs_disp = probs_disp / 2 + 0.5 # shift to 0.5-1 range
    s = mprompt.viz.colorize(words, probs_disp, title=expl, subtitle=prompt)
    
    # viz
    display(HTML(s))
    story_running += ' ' + s

with open('../results/story_running.html', 'w') as f:
    f.write(story_running)

### Heatmap of story <> explanation matches

In [ ]:
n = len(expls)
scores = np.zeros((n, n))
for i in tqdm(range(n)):
    expl = expls[i]
    for j in range(n):
        text = paragraphs[j].lower()
        words = text.split()

        ngrams = imodelsx.util.generate_ngrams_list(text, ngrams=3)
        ngrams = [words[0], words[0] + ' ' + words[1]] + ngrams

        # validator-based viz
        probs = np.array(val.validate_w_scores(expl, ngrams)) > 0.5
        scores[i, j] = probs.mean()

In [ ]:
s = scores
# s = scipy.special.softmax(scores, axis=1)
# s = (s - s.min()) / (s.max() - s.min())
plt.style.use('dark_background')
plt.figure(figsize=(6, 5))
plt.imshow(s)
plt.xticks(range(n), expls, rotation=90)
plt.yticks(range(n), expls)
plt.ylabel('Explanation for generation')
plt.xlabel('Explanation for matching')
plt.colorbar(label='Fraction of matching ngrams')
plt.tight_layout()
plt.show()

# Test modules on the generated stories

In [ ]:
n = len(expls)
scores = np.zeros((n, n))
scores_max = np.zeros((n, n))
all_scores = {}
all_ngrams = {}
for i in tqdm(range(n)):
    # expl = expls[i]
    (expl, subject, voxel_num_best) = voxels[i]
    mod = fMRIModule(voxel_num_best=voxel_num_best, subject=subject)
    ngrams_list = []
    ngrams_scores_list = []
    for j in range(n):
        text = paragraphs[j].lower()
        words = text.split()
        ngrams_story = imodelsx.util.generate_ngrams_list(text, ngrams=3)
        ngrams_story = [words[0], words[0] + ' ' + words[1]] + ngrams_story
        ngrams_list.append(ngrams_story)

        # get mean score for each story
        ngrams_scores_story = mod(ngrams_story)
        ngrams_scores_list.append(ngrams_scores_story)
        scores[i, j] = ngrams_scores_story.mean()
        scores_max[i, j] = ngrams_scores_story.max()
    
    all_scores[i] = deepcopy(ngrams_scores_list)
    all_ngrams[i] = deepcopy(ngrams_list)

In [ ]:
s = scores
s = scipy.special.softmax(scores, axis=1)
# s = (s - s.min()) / (s.max() - s.min())
plt.style.use('dark_background')
plt.figure(figsize=(6, 5))
plt.imshow(s)
plt.xticks(range(n), expls, rotation=90)
plt.yticks(range(n), expls)
plt.ylabel('Explanation for generation')
plt.xlabel('Explanation of voxel used for evaluation')
plt.colorbar(label='Mean voxel response')
plt.tight_layout()
plt.show()

### Module responses for single story

In [5]:
pd.set_option('display.max_rows', 120)
for i in range(1): #len(voxels)):
    row = rows_check.iloc[i]
    display(row[['subject', 'module_num', 'top_explanation_init_strs', 'explanation_init_ngrams', 'top_ngrams_module_correct']])
    mod = fMRIModule(voxel_num_best=row.module_num, subject=row.subject)
    # show all rows
    # display(
    #     pd.DataFrame.from_dict({
    #         'score': all_scores[i][i],
    #         'ngram': all_ngrams[i][i],
    #     }).sort_values('score', ascending=False).head(10)
    # )

subject                                                                  UTS01
module_num                                                                   7
top_explanation_init_strs                                   movement or action
explanation_init_ngrams      [turned and walked, stopped midstride, slowly ...
top_ngrams_module_correct    [turned and walked, stopped midstride, slowly ...
Name: 3, dtype: object

In [6]:
x = row['explanation_init_ngrams']

In [7]:
out = mod(x, return_all=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 75/75 [00:10<00:00,  6.83it/s]


In [8]:
p = out[:, row.module_num]

In [ ]:
out